<a href="https://colab.research.google.com/github/AnsimovDaniel/Manifest/blob/main/Manifest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import scipy.io

import numpy as np

from sklearn.metrics.pairwise import euclidean_distances


def polynomial_kernel_matrix(x, degree, constant):
    return (np.dot(x.T, x) + constant) ** degree
def construct_kernel(X, y, kernel_scale_factor, kernel_type='laplacian', poly_degree=2, poly_constant=1):
    label = list(set(y))
    x_1 = X[np.where(y == label[0])]
    x_2 = X[np.where(y == label[1])]

    if kernel_type == 'rbf':
        K1_dis = euclidean_distances(np.transpose(x_1))
        K2_dis = euclidean_distances(np.transpose(x_2))

        epsilon1 = kernel_scale_factor * np.median(K1_dis[~np.eye(K1_dis.shape[0], dtype=bool)])
        epsilon2 = kernel_scale_factor * np.median(K2_dis[~np.eye(K2_dis.shape[0], dtype=bool)])

        K1 = np.exp(-K1_dis / epsilon1)
        K2 = np.exp(-K2_dis / epsilon2)
    elif kernel_type == 'laplacian':
        K1_dis = euclidean_distances(np.transpose(x_1))
        K2_dis = euclidean_distances(np.transpose(x_2))

        epsilon1 = kernel_scale_factor * np.median(K1_dis[~np.eye(K1_dis.shape[0], dtype=bool)])
        epsilon2 = kernel_scale_factor * np.median(K2_dis[~np.eye(K2_dis.shape[0], dtype=bool)])

        K1 = np.exp(-np.sqrt(K1_dis) / epsilon1)
        K2 = np.exp(-np.sqrt(K2_dis) / epsilon2)


    elif kernel_type == 'polynomial':

        K1 = polynomial_kernel_matrix(x_1, poly_degree, poly_constant)

        K2 = polynomial_kernel_matrix(x_2, poly_degree, poly_constant)

    else:
        raise ValueError("Invalid kernel type. Choose 'rbf', 'sigmoid', or 'laplacian'.")

    return K1, K2


def calc_tol(matrix,var_type='float64',energy_tol = 0 ):
        tol = np.max(matrix) * len(matrix) * np.core.finfo(var_type).eps
        tol2 = np.sqrt(np.sum(matrix**2)*energy_tol)
        tol = np.max([tol,tol2])

        return tol

def spsd_geodesics(G1,G2,p=0.5,r=None,eigVecG1=None,eigValG1=None,eigVecG2=None,eigValG2=None):

    if eigVecG1 is None:
        eigValG1,eigVecG1 =np.linalg.eigh(G1)
    if eigVecG2 is None:
        eigValG2,eigVecG2 =np.linalg.eigh(G2)

    if r is None:
        tol = calc_tol(eigValG1)
        rank_G1 = len(np.abs(eigValG1)[np.abs(eigValG1)>tol])

        tol = calc_tol(eigValG2)
        rank_G2 = len(np.abs(eigValG2)[np.abs(eigValG2)>tol])

        r = min(rank_G1,rank_G2)

    maxIndciesG1 = np.flip(np.argsort(np.abs(eigValG1))[-r:],[0])
    V1 = eigVecG1[:,maxIndciesG1]
    lambda1 = eigValG1[maxIndciesG1]

    maxIndciesG2 = np.flip(np.argsort(np.abs(eigValG2))[-r:],[0])
    V2 = eigVecG2[:,maxIndciesG2]
    lambda2 = eigValG2[maxIndciesG2]

    #lapack_driver='gesvd' is more stable while lapack_driver='gesdd' is more fast
    try:
        O2,sigma,O1T =np.linalg.svd(V2.T@V1)
    except:
        O2,sigma,O1T =scipy.linalg.svd(V2.T@V1,lapack_driver='gesvd')

    O1 = O1T.T

    sigma[sigma<-1] =-1
    sigma[sigma>1] =1
    theta = np.arccos(sigma)

    U1 = V1@O1
    R1 = O1.T@np.diag(lambda1)@O1

    U2 = V2@O2
    R2 = O2.T@np.diag(lambda2)@O2

    tol = calc_tol(sigma)
    valid_ind = np.where(np.abs(sigma-1)>tol)
    pinv_sin_theta = np.zeros(theta.shape)
    pinv_sin_theta[valid_ind]=1/np.sin(theta[valid_ind])

    UG1G2 = U1@np.diag(np.cos(theta*p))+(np.eye(G1.shape[0])-U1@U1.T)@U2@np.diag(pinv_sin_theta)@np.diag(np.sin(theta*p))

    return UG1G2,R1,R2,O1,lambda1


def get_operators(K1,K2,use_spsd=True):

    if  use_spsd:
        eigValK1,eigVecK1 =np.linalg.eigh(K1)
        tol = calc_tol(eigValK1)
        rank_K1 = len(np.abs(eigValK1)[np.abs(eigValK1)>tol])

        eigValK2,eigVecK2 =np.linalg.eigh(K2)
        tol = calc_tol(eigValK2)
        rank_K2 = len(np.abs(eigValK2)[np.abs(eigValK2)>tol])

        # create SPSD Mean operator M
        min_rank = min(rank_K1,rank_K2)
        UK1K2,RK1,RK2,OK1,lambdaK1 = spsd_geodesics(K1,K2,p=0.5,r=min_rank,eigVecG1=eigVecK1,eigValG1=eigValK1,eigVecG2=eigVecK2,eigValG2=eigValK2)

        RK1PowerHalf = OK1.T@np.diag(np.sqrt(lambdaK1))@OK1
        RK1PowerMinusHalf = OK1.T@np.diag(1/np.sqrt(lambdaK1))@OK1
        e, v = np.linalg.eigh(RK1PowerMinusHalf@RK2@RK1PowerMinusHalf)
        e[e<0]=0
        RK1K2 = RK1PowerHalf@v@np.diag(np.sqrt(e))@v.T@RK1PowerHalf

        M = UK1K2@RK1K2@UK1K2.T

        eigValM,eigVecM =np.linalg.eigh(M)
        tol = calc_tol(eigValM)
        rank_M = len(np.abs(eigValM)[np.abs(eigValM)>tol])

        # create SPSD Difference operator D
        min_rank = min(rank_K1,rank_M)
        UMK1,RM,RK1,OM,lambdaM = spsd_geodesics(M,K1,p=1,r=min_rank,eigVecG1=eigVecM,eigValG1=eigValM,eigVecG2=eigVecK1,eigValG2=eigValK1)

        RMPowerHalf = OM.T@np.diag(np.sqrt(lambdaM))@OM
        RMPowerMinusHalf = OM.T@np.diag(1/np.sqrt(lambdaM))@OM
        e,v = np.linalg.eigh(RMPowerMinusHalf@RK1@RMPowerMinusHalf)
        tol = calc_tol(e)
        e[e<tol]=tol
        logarithmic = RMPowerHalf@v@np.diag(np.log(e))@v.T@RMPowerHalf

        D= UMK1@logarithmic@UMK1.T

    else:   #SPD form
        # create SPD Mean operator M
        K1 = K1 +np.eye(K1.shape[0])*np.core.finfo('float64').eps*2
        K2 = K2 +np.eye(K2.shape[0])*np.core.finfo('float64').eps*2

        eigValK1,eigVecK1 =np.linalg.eigh(K1)
        tol = calc_tol(eigValK1)
        rank_K1 = len(np.abs(eigValK1)[np.abs(eigValK1)>tol])

        K1PowerHalf =eigVecK1@np.diag(np.sqrt(eigValK1))@eigVecK1.T
        K1PowerMinusHalf =eigVecK1@np.diag(1/np.sqrt(eigValK1))@eigVecK1.T
        e, v = np.linalg.eigh(K1PowerMinusHalf@K2@K1PowerMinusHalf)
        e[e<0]=0
        M = K1PowerHalf@v@np.diag(np.sqrt(e))@v.T@K1PowerHalf

        eigValM,eigVecM =np.linalg.eigh(M)
        tol = calc_tol(eigValM)
        rank_M = len(np.abs(eigValM)[np.abs(eigValM)>tol])

        # create SPD Difference operator D
        MPowerHalf =eigVecM@np.diag(np.sqrt(eigValM))@eigVecM.T
        MPowerMinusHalf =eigVecM@np.diag(1/np.sqrt(eigValM))@eigVecM.T
        e,v = np.linalg.eigh(MPowerMinusHalf@K1@MPowerMinusHalf)
        tol = calc_tol(e)
        e[e<tol]=tol
        D = MPowerHalf@v@np.diag(np.log(e))@v.T@MPowerHalf

    return M,D

def compute_manifest_score(D):

    eigValD,eigVecD =np.linalg.eigh(D)

    eigVec_norm = eigVecD**2
    score = eigVec_norm@np.abs(eigValD)

    return score


def ManiFeSt(X,y,kernel_scale_factor=1,use_spsd=True):

    K1,K2 = construct_kernel(X,y,kernel_scale_factor)

    M,D = get_operators(K1,K2,use_spsd=use_spsd)

    score = compute_manifest_score(D)
    idx = np.argsort(score, 0)[::-1]

    ##eig_vecs
    eigValM,eigVecM =np.linalg.eigh(M)
    eigValD,eigVecD =np.linalg.eigh(D)

    sorted_indexes = np.argsort(np.abs(eigValM))[::-1]
    eigVecM = eigVecM[:,sorted_indexes]
    eigValM = eigValM[sorted_indexes]
    sorted_indexes = np.argsort(np.abs(eigValD))[::-1]
    eigVecD = eigVecD[:,sorted_indexes]
    eigValD = eigValD[sorted_indexes]

    eig_vecs = (eigVecD,eigValD,eigVecM,eigValM)

    return score, idx, eig_vecs

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('/content/drive/MyDrive/Breast_GSE10797.csv')
print(df.head())

mapping = {'cancer_epithelial': 'cancer', 'cancer_stroma': 'cancer'}

# Replace values in the 'your_column_name' column
df['type'] = df['type'].replace(mapping)


X = df.drop(['type', 'samples'], axis = 1).values
X = X[:, :8000]
Y = df['type'].values

print(df['type'].unique())


num_of_samples = X.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=30, shuffle = True)


   samples               type  1007_s_at   1053_at    117_at    121_at  \
0      671  cancer_epithelial   6.421287  3.161197  1.965993  2.600064   
1      673  cancer_epithelial   6.715620  3.131431  2.624435  3.340376   
2      675  cancer_epithelial   7.019017  3.080859  2.977302  2.692181   
3      677  cancer_epithelial   7.595891  2.863032  2.499157  3.004193   
4      679  cancer_epithelial   7.312846  2.498788  2.064275  3.180191   

   1255_g_at   1294_at   1316_at   1320_at  ...  AFFX-r2-Ec-bioD-3_at  \
0   2.157899  3.153408  2.012364  1.976855  ...             12.005226   
1   2.214897  2.998384  2.377003  2.219821  ...             12.929805   
2   2.023505  3.547079  2.354219  2.319934  ...             12.604099   
3   2.033855  3.405192  2.346281  2.261241  ...             13.134787   
4   2.011954  2.919717  2.099707  2.058700  ...             11.984974   

   AFFX-r2-Ec-bioD-5_at  AFFX-r2-P1-cre-3_at  AFFX-r2-P1-cre-5_at  \
0             12.476655            14.271008   

In [ ]:
use_spsd = True  #False - use SPD form  - default is SPSD, MNIST is SPSD since there are blank pixels
kernel_scale_factor = 1 # The RBF kernel scale is typically set to the median of the Euclidean distances up to some scalar defiend by kernel_scale_factor ,  default value 1
score, idx, eig_vecs = ManiFeSt(X_train,y_train,kernel_scale_factor=kernel_scale_factor,use_spsd=use_spsd)  #use_spsd=use_spsd

In [2]:
num_of_features = [10, 20, 50, 100, 150, 200, 300]
num_cv_folds = 5  # Number of cross-validation folds
avg_cv_scores = []

for n in num_of_features:

    # Select the top 'n' features using sorted indices 'idx'
    feature_subset = X[:, idx[:n]]
    print(feature_subset.shape)

    # Initialize StratifiedKFold for stratified cross-validation
    skf = StratifiedKFold(n_splits=num_cv_folds, shuffle=True, random_state=42)

    # Train SVM Classifier and compute cross-validation scores
    svm_classifier = SVC(kernel='linear')
    cv_scores = cross_val_score(svm_classifier, feature_subset, Y, cv=skf)

    # Calculate average cross-validation score for this number of features
    avg_cv_score = np.mean(cv_scores)
    avg_cv_scores.append(avg_cv_score)
    print("Num of features:", n)
    print("Average Cross-Validation Accuracy:", avg_cv_score)


NameError: name 'X' is not defined

In [ ]:
plt.figure()
plt.plot(num_of_features, avg_cv_scores)
plt.xlabel('Number of features')
plt.ylabel('Accuracy, %')
plt.grid('on')

In [8]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import cv2
import os
from PIL import Image
from sklearn.model_selection  import train_test_split
import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec